In [1]:
import pandas as pd
import sqlite3

### Reading the cleaned_data from MS Excel via pandas dataframe

In [2]:
data = pd.read_csv("StudentUsageData.csv", header = 0)

In [3]:
data.head()

,Date,Month,Day_of_week,userId,first_name,last_name,full_name,Course_Name,ClassName,Duration,Grade,School_Name
0,02-Aug-22,Aug,Tuesday,101,Mownesh,M,Mownesh M,Bird Feeder,NaN,160,3,ABC
1,02-Aug-22,Aug,Tuesday,101,Mownesh,M,Mownesh M,Bird Feeder,NaN,185,3,ABC
2,02-Aug-22,Aug,Tuesday,101,Mownesh,M,Mownesh M,Bird Feeder,NaN,110,3,ABC
3,02-Aug-22,Aug,Tuesday,107,Girish,K S,Girish K S,Tips and Tricks- Addition and Subtraction,NaN,100,4,VVCE
4,02-Aug-22,Aug,Tuesday,107,Girish,K S,Girish K S,Wind Vane,NaN,195,4,VVCE


### Converting the dataframe to SQL table via sqlite3

In [4]:
db_conn = sqlite3.connect("teaching_org_da.db")
c = db_conn.cursor()

In [5]:
c.execute(
    """
   CREATE TABLE data (
        Date TEXT,
        Month TEXT,
        Day_of_week TEXT,
        userId INTEGER,
        first_name TEXT,
        last_name TEXT,
        full_name TEXT,
        Course_Name TEXT,
        ClassName TEXT,
        Duration INTEGER,
        Grade INTEGER,
        School_Name TEXT
);   
    """
)

In [6]:
data.to_sql('data', db_conn, if_exists='append', index=False)

In [7]:
pd.read_sql("select * from data",db_conn)

,Date,Month,Day_of_week,userId,first_name,last_name,full_name,Course_Name,ClassName,Duration,Grade,School_Name
0,02-Aug-22,Aug,Tuesday,101,Mownesh,M,Mownesh M,Bird Feeder,None,160,3,ABC
1,02-Aug-22,Aug,Tuesday,101,Mownesh,M,Mownesh M,Bird Feeder,None,185,3,ABC
2,02-Aug-22,Aug,Tuesday,101,Mownesh,M,Mownesh M,Bird Feeder,None,110,3,ABC
3,02-Aug-22,Aug,Tuesday,107,Girish,K S,Girish K S,Tips and Tricks- Addition and Subtraction,None,100,4,VVCE
4,02-Aug-22,Aug,Tuesday,107,Girish,K S,Girish K S,Wind Vane,None,195,4,VVCE
...,...,...,...,...,...,...,...,...,...,...,...,...
998,17-Jan-23,Jan,Tuesday,863,Yashika,SM,Yashika SM,Frogs - The Outstanding Amphibians,None,0,4,VVCE
999,17-Jan-23,Jan,Tuesday,479,Prasad,M,Prasad M,Sarale Varase in Maya Malava Gowla - Part 1,None,155,3,ABC
1000,18-Jan-23,Jan,Wednesday,935,Yathin,SD,Yathin SD,Digital Out - LEDs,None,30,9,VVCE
1001,18-Jan-23,Jan,Wednesday,941,Puneeth,SM,Puneeth SM,WHO AM I - Peep Into A Teen's Life,None,270,3,TQH


In [8]:
pd.read_sql("select Day_of_week, sum(Duration) as Total_duration_day, avg(Duration) as Average_stream_time from data group by Day_of_week order by Total_duration_day DESC",db_conn)

,Day_of_week,Total_duration_day,Average_stream_time
0,Tuesday,28240,183.376623
1,Thursday,25950,136.578947
2,Monday,22545,180.360000
3,Friday,19945,145.583942
4,Sunday,18895,116.635802
5,Wednesday,14150,126.339286
6,Saturday,13135,106.788618


In [9]:
pd.read_sql("select Month, count(distinct full_name) as Number_of_users, sum(Duration) as Total_duration_day, avg(Duration) as Average_stream_time from data group by Month order by Total_duration_day DESC",db_conn)

,Month,Number_of_users,Total_duration_day,Average_stream_time
0,Oct,89,42065,117.500000
1,Sep,85,35820,158.495575
2,Aug,43,24135,202.815126
3,Nov,43,19505,121.906250
4,Dec,31,12270,129.157895
5,Jan,18,9065,201.444444


In [10]:
pd.read_sql("select full_name, School_Name, Grade, sum(duration) as Binge_watching_duration from data group by full_name order by Binge_watching_duration DESC",db_conn)

,full_name,School_Name,Grade,Binge_watching_duration
0,Prerana B,ABC,8,9940
1,Gagan V,ABC,4,8050
2,Neha sree,ABC,8,8010
3,Jhanavi C,VVCE,9,5290
4,Nithin D,TQH,4,5095
...,...,...,...,...
153,Likith P,ABC,6,20
154,Keerthan YM,VVCE,6,20
155,Sai Jeevan,VVCE,4,5
156,Mohammed Mohammed,ABC,6,5


In [11]:
pd.read_sql("select Course_Name, sum(duration) as highest_watched_course from data group by Course_Name order by highest_watched_course DESC limit 5",db_conn)

,Course_Name,highest_watched_course
0,Watercolours - Let's get started,8760
1,Money Management,6875
2,Greetings and Wishes,6745
3,Tips and Tricks- Addition and Subtraction,5805
4,Ukulele - Part 2,5135


In [12]:
pd.read_sql("select Course_Name, sum(duration) as lowest_watched_course from data group by Course_Name having lowest_watched_course > 0 order by lowest_watched_course limit 5",db_conn)

,Course_Name,lowest_watched_course
0,Magic of Blending With Cotton Buds and Tissue ...,5
1,Basic Shading,15
2,Awesome Animals with Paper Plates,20
3,Magic of Blending With Paper Stumps and Fingers,25
4,Global finance,30


In [13]:
pd.read_sql("select Course_Name, sum(duration) from data group by Course_Name having sum(duration) = 0",db_conn)

,Course_Name,sum(duration)
0,A part of a whole,0
1,App building Platform overview,0
2,Basics of HTML,0
3,Math aptitude - 1,0
4,My Portfolio,0
5,Number hunt,0
6,Platform walkthrough and sprite objects,0
7,Post-A-Mail,0
8,Riddle App,0
9,Spanish Translator,0


In [14]:
pd.read_sql("select count(distinct Course_Name), count(distinct Grade), count(distinct School_Name) from data",db_conn)

,count(distinct Course_Name),count(distinct Grade),count(distinct School_Name)
0,125,7,3


In [15]:
pd.read_sql("select School_Name, count(distinct full_name) as number_of_students, sum(duration) as total_duration, avg(duration) as average_duration from data group by School_Name order by total_duration DESC",db_conn)

,School_Name,number_of_students,total_duration,average_duration
0,ABC,55,62885,155.655941
1,TQH,50,40345,137.227891
2,VVCE,53,39630,129.934426


In [16]:
pd.read_sql("select Grade, count(distinct full_name) as number_of_students, sum(duration) as total_duration, avg(duration) as average_duration from data group by Grade",db_conn)

,Grade,number_of_students,total_duration,average_duration
0,3,22,17910,118.609272
1,4,24,26395,174.801325
2,5,25,15705,113.804348
3,6,22,10400,96.296296
4,7,22,18730,137.720588
5,8,21,31940,180.451977
6,9,22,21780,153.380282


In [17]:
pd.read_sql("select Grade, School_Name, sum(duration) as total_duration from data group by School_Name, Grade",db_conn)

,Grade,School_Name,total_duration
0,3,ABC,8950
1,4,ABC,11670
2,5,ABC,9365
3,6,ABC,1450
4,7,ABC,5855
5,8,ABC,19760
6,9,ABC,5835
7,3,TQH,4325
8,4,TQH,11970
9,5,TQH,3610


In [18]:
pd.read_sql("select userId, full_name, max(Date) as lastest_login_date, min(Date) as login_date from data group by full_name",db_conn)

,userId,full_name,lastest_login_date,login_date
0,245,ABINAYA R,16-Nov-22,05-Aug-22
1,383,ASHWIN B,04-Sep-22,03-Sep-22
2,227,Abhi Ram,27-Oct-22,05-Aug-22
3,299,Abhinav SM,15-Oct-22,03-Oct-22
4,299,Abhinav T,08-Aug-22,08-Aug-22
...,...,...,...,...
153,815,Yasheshwini MK,30-Sep-22,02-Jan-23
154,863,Yashika SM,17-Jan-23,02-Oct-22
155,935,Yathin SD,18-Jan-23,02-Dec-22
156,461,Yeshwanth Kumar,17-Nov-22,12-Sep-22


In [19]:
pd.read_sql("select count(distinct userId) as num_IDs, count(distinct full_name) as num_students from data",db_conn)

,num_IDs,num_students
0,150,158
